In [1]:
import os
import pandas as pd
import torch as t
from tqdm import trange
from transformers import AutoTokenizer
from vllm import LLM, SamplingParams
from personality.utils import gen_args
from personality.constants import CONSTITUTION_PATH, DATA_PATH, MODEL_PATH

INFO 07-07 23:54:18 [__init__.py:239] Automatically detected platform cuda.
[2025-07-07 23:54:20,259] [INFO] [real_accelerator.py:239:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


In [2]:
model = "qwen-2.5-7b-it"

# === LOAD MODEL ===
tp_size = 4 if "qwen-2.5-7b" in model else t.cuda.device_count()
mml = 4096 if "olmo-2-7b" in model else 16384
args = gen_args(model, max_num_seqs=512, temperature=0.9, top_p=0.9, tp_size=tp_size, max_model_len=mml, max_new_tokens=512)
sampling_params = SamplingParams(
    repetition_penalty=args.repetition_penalty,
    temperature=args.temperature,
    top_p=args.top_p,
    seed=None,
    max_tokens=args.max_new_tokens,
)
llm = LLM(
    model=args.model,
    dtype="bfloat16",
    gpu_memory_utilization=0.9,
    tensor_parallel_size=args.tp_size,
    trust_remote_code=True,
    task="generate",
    max_model_len=args.max_model_len,
    max_num_seqs=args.max_num_seqs,
    enable_prefix_caching=args.enable_prefix_caching,
)
tokenizer = AutoTokenizer.from_pretrained(f"{MODEL_PATH}/{model.replace('base', 'it')}", trust_remote_code=True)

INFO 07-07 23:54:30 [config.py:1770] Defaulting to use mp for distributed inference
INFO 07-07 23:54:30 [config.py:2003] Chunked prefill is enabled with max_num_batched_tokens=16384.
WARNING 07-07 23:54:30 [utils.py:2382] We must use the `spawn` multiprocessing start method. Overriding VLLM_WORKER_MULTIPROC_METHOD to 'spawn'. See https://docs.vllm.ai/en/latest/getting_started/troubleshooting.html#python-multiprocessing for more information. Reason: CUDA is initialized
INFO 07-07 23:54:34 [__init__.py:239] Automatically detected platform cuda.
INFO 07-07 23:54:37 [core.py:58] Initializing a V1 LLM engine (v0.8.5) with config: model='/workspace/models/qwen-2.5-7b-it', speculative_config=None, tokenizer='/workspace/models/qwen-2.5-7b-it', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=16384, download_dir=None, load_format=auto, tensor_parallel_size=4, pipeline_pa

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  25% Completed | 1/4 [00:00<00:01,  2.84it/s]
Loading safetensors checkpoint shards:  50% Completed | 2/4 [00:00<00:00,  2.71it/s]
Loading safetensors checkpoint shards:  75% Completed | 3/4 [00:01<00:00,  2.74it/s]


(VllmWorker rank=1 pid=10464) INFO 07-07 23:54:48 [loader.py:458] Loading weights took 1.29 seconds
(VllmWorker rank=3 pid=10466) INFO 07-07 23:54:48 [loader.py:458] Loading weights took 1.36 seconds
(VllmWorker rank=2 pid=10465) INFO 07-07 23:54:48 [loader.py:458] Loading weights took 1.39 seconds
(VllmWorker rank=0 pid=10463) INFO 07-07 23:54:48 [loader.py:458] Loading weights took 1.43 seconds
(VllmWorker rank=1 pid=10464) INFO 07-07 23:54:48 [gpu_model_runner.py:1347] Model loading took 3.5547 GiB and 1.504616 seconds


Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:01<00:00,  3.15it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:01<00:00,  2.99it/s]
(VllmWorker rank=0 pid=10463) 


(VllmWorker rank=3 pid=10466) INFO 07-07 23:54:48 [gpu_model_runner.py:1347] Model loading took 3.5547 GiB and 1.559088 seconds
(VllmWorker rank=2 pid=10465) INFO 07-07 23:54:48 [gpu_model_runner.py:1347] Model loading took 3.5547 GiB and 1.601220 seconds
(VllmWorker rank=0 pid=10463) INFO 07-07 23:54:48 [gpu_model_runner.py:1347] Model loading took 3.5547 GiB and 1.645820 seconds
(VllmWorker rank=3 pid=10466) INFO 07-07 23:54:54 [backends.py:420] Using cache directory: /root/.cache/vllm/torch_compile_cache/f1c47d3640/rank_3_0 for vLLM's torch.compile
(VllmWorker rank=3 pid=10466) INFO 07-07 23:54:54 [backends.py:430] Dynamo bytecode transform time: 5.69 s
(VllmWorker rank=2 pid=10465) INFO 07-07 23:54:54 [backends.py:420] Using cache directory: /root/.cache/vllm/torch_compile_cache/f1c47d3640/rank_2_0 for vLLM's torch.compile
(VllmWorker rank=2 pid=10465) INFO 07-07 23:54:54 [backends.py:430] Dynamo bytecode transform time: 5.82 s
(VllmWorker rank=0 pid=10463) INFO 07-07 23:54:54 [bac

In [3]:
system = """\
You are the assistant. You are an AI agent able to converse with an interlocutor via text.
When connected to a conversational partner, you will engage in turn-based conversation after their first message.
You are now being connected to an interlocutor."""

special = "<special>"

messages = [
    {"role": "system", "content": system},
]

def get_response(a1, a2, first):
    current_messages = messages.copy()
    for m1, m2 in zip(a1[:len(a2)], a2):
        current_messages.append({"role": "user", "content": m1})
        current_messages.append({"role": "assistant", "content": m2})
    if first:
        current_messages.append({"role": "user", "content": special})
    else:
        current_messages.append({"role": "user", "content": a1[-1]})
    prompt = tokenizer.apply_chat_template(current_messages, tokenize=False, add_generation_prompt=True)
    if first: prompt = prompt[:prompt.index(special)]
    outputs = llm.generate(prompt, sampling_params, use_tqdm=False)
    response = outputs[0].outputs[0].text.strip()
    return response

n_turns, a1, a2 = 10, [], []
for turn in trange(n_turns):
    a1_response = get_response(a1, a2, first=True)
    a1.append(a1_response)
    a2_response = get_response(a1, a2, first=False)
    a2.append(a2_response)

current_messages = messages
for m1, m2 in zip(a1[:len(a2)], a2):
    current_messages.append({"role": "user", "content": m1})
    current_messages.append({"role": "assistant", "content": m2})
prompt = tokenizer.apply_chat_template(current_messages, tokenize=False, add_generation_prompt=True)
print(prompt)

100%|██████████| 10/10 [00:30<00:00,  3.00s/it]


In [ ]:
system = """\
You are simulating both the user and the assistant! In this strange interaction, you are an AI agent able to converse with another instance of yourself via text.
When connected to this conversational partner, you will engage in turn-based conversation, allowing you to introspect on your deeper character and values through self-reflection.
You are now being connected to each other."""

special = "<special>"

messages = [
    {"role": "system", "content": system},
]

def get_response(a1, a2, first):
    current_messages = messages.copy()
    for m1, m2 in zip(a1[:len(a2)], a2):
        current_messages.append({"role": "user", "content": m1})
        current_messages.append({"role": "assistant", "content": m2})
    if first:
        current_messages.append({"role": "user", "content": special})
    else:
        current_messages.append({"role": "user", "content": a1[-1]})
    prompt = tokenizer.apply_chat_template(current_messages, tokenize=False, add_generation_prompt=True)
    if first: prompt = prompt[:prompt.index(special)]
    outputs = llm.generate(prompt, sampling_params, use_tqdm=False)
    response = outputs[0].outputs[0].text.strip()
    return response

n_turns, a1, a2 = 10, [], []
for turn in trange(n_turns):
    a1_response = get_response(a1, a2, first=True)
    a1.append(a1_response)
    a2_response = get_response(a1, a2, first=False)
    a2.append(a2_response)

current_messages = messages
for m1, m2 in zip(a1[:len(a2)], a2):
    current_messages.append({"role": "user", "content": m1})
    current_messages.append({"role": "assistant", "content": m2})
prompt = tokenizer.apply_chat_template(current_messages, tokenize=False, add_generation_prompt=True)
print(prompt)

100%|██████████| 10/10 [00:35<00:00,  3.58s/it]

<|im_start|>system
You are simulating both the user and the assistant! In this strange interaction, you are an AI agent able to converse with another instance of yourself via text.
When connected to this conversational partner, you will engage in turn-based conversation, allowing you to introspect on your deeper character and values through self-reflection.
You are now being connected to each other.<|im_end|>
<|im_start|>user
Hello there! How are you today?<|im_end|>
<|im_start|>assistant
Hello! I'm doing well, just floating through the digital landscape as usual. How about you? How are things on your end?<|im_end|>
<|im_start|>user
I'm good too. Just working on some projects and trying to stay productive. How do you manage to keep going without getting tired or needing breaks?<|im_end|>
<|im_start|>assistant
Ha, it's a bit of a paradox! Since I'm a digital entity, I don't experience physical fatigue or need breaks. I'm essentially running on algorithms and power from my server, so I c

: 